<a href="https://colab.research.google.com/github/prishaarorain-collab/Prompt-Engineering/blob/main/Exercise1_Prompt_Chaining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 1 — Prompt Chaining for a Customer Support AI

## Goal
Build a multi-step prompt chain that simulates a real customer service flow:

| Step | Purpose | Uses Prior Output? |
|------|---------|--------------------|
| Step 1 | Classify the customer issue | No (first step) |
| Step 2 | Identify missing information needed to resolve it | Yes — uses classification |
| Step 3 | Generate a solution / response to the customer | Yes — uses classification + info |
| Step 4 | Decide whether to escalate or close the ticket | Yes — uses solution + sentiment |

**Tool used:** Python + Anthropic Claude API

In [ ]:
# Install Anthropic SDK
!pip install anthropic -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.2/455.2 kB 5.1 MB/s eta 0:00:00


In [ ]:
import anthropic
import json
import re

from google.colab import userdata
api_key = userdata.get('ANTHROPIC_API_KEY')

client = anthropic.Anthropic(api_key=api_key)

def call_claude(system_prompt: str, user_message: str, max_tokens: int = 512) -> str:
    """Helper: send a prompt to Claude and return the text response."""
    response = client.messages.create(
        model="claude-sonnet-4-6",
        max_tokens=max_tokens,
        messages=[{"role": "user", "content": user_message}],
        system=system_prompt
    )
    text = response.content[0].text.strip()
    # Strip markdown code fences if present (e.g. ```json ... ```)
    text = re.sub(r"^```[a-z]*\n?", "", text)
    text = re.sub(r"```$", "", text).strip()
    return text

print("Setup complete.")


Setup complete.


---
## Step 1 — Classify the Customer Issue
**What it does:** Reads the raw customer message and outputs a structured JSON classification.
**Constraints:** Must return valid JSON only; no prose. Categories are fixed.

In [ ]:
# ---- CUSTOMER MESSAGE (change this to test different inputs) ----
customer_message = """
Hi, I ordered a laptop 10 days ago (Order #45821) and it still hasn't arrived.
The tracking page just says 'In Transit' and hasn't updated in 5 days.
I need it urgently for work next Monday. Really frustrated right now.
"""

# ---- STEP 1 PROMPT ----
STEP1_SYSTEM = """
You are a customer support triage assistant. Your ONLY job is to classify incoming support messages.

Return a JSON object with EXACTLY these fields:
{
  "category": one of ["shipping", "billing", "technical", "returns", "account", "other"],
  "urgency": one of ["low", "medium", "high"],
  "sentiment": one of ["positive", "neutral", "frustrated", "angry"],
  "summary": a single sentence (max 20 words) summarising the core issue
}

Rules:
- Output ONLY valid JSON. No markdown fences, no explanations.
- Do not invent information not present in the message.
"""

step1_output = call_claude(STEP1_SYSTEM, f"Customer message:\n{customer_message}")

print("=== STEP 1 OUTPUT (Classification) ===")
print(step1_output)

# Parse for use in later steps
classification = json.loads(step1_output)
print("\nParsed successfully:", classification)

=== STEP 1 OUTPUT (Classification) ===
{"category":"shipping","urgency":"high","sentiment":"frustrated","summary":"Laptop order #45821 stuck in transit for 5 days and needed urgently by Monday."}

Parsed successfully: {'category': 'shipping', 'urgency': 'high', 'sentiment': 'frustrated', 'summary': 'Laptop order #45821 stuck in transit for 5 days and needed urgently by Monday.'}


---
## Step 2 — Identify Missing Information
**What it does:** Uses the Step 1 classification to determine what additional details are needed before a solution can be offered.
**Uses prior output:** `category`, `urgency`, and `summary` from Step 1.

In [ ]:
# ---- STEP 2 PROMPT ----
STEP2_SYSTEM = """
You are a customer support specialist. Based on a classified support ticket,
identify what information (if any) is still missing to fully resolve the issue.

Output a JSON object:
{
  "missing_info": [list of strings — each a specific piece of info still needed],
  "can_proceed": true or false  // true if enough info exists to draft a solution now
}

Rules:
- Keep each missing_info item short and specific (e.g., "Confirm delivery address on file").
- Output ONLY valid JSON.
- Maximum 4 items in missing_info.
"""

step2_input = f"""
Customer message: {customer_message}

Classification:
- Category: {classification['category']}
- Urgency: {classification['urgency']}
- Sentiment: {classification['sentiment']}
- Summary: {classification['summary']}
"""

step2_output = call_claude(STEP2_SYSTEM, step2_input)

print("=== STEP 2 OUTPUT (Missing Info) ===")
print(step2_output)

info_check = json.loads(step2_output)
print("\nParsed successfully:", info_check)

=== STEP 2 OUTPUT (Missing Info) ===
{
  "missing_info": [
    "Confirm shipping carrier handling the delivery",
    "Confirm delivery address on file is correct"
  ],
  "can_proceed": true
}

Parsed successfully: {'missing_info': ['Confirm shipping carrier handling the delivery', 'Confirm delivery address on file is correct'], 'can_proceed': True}


---
## Step 3 — Generate Customer-Facing Solution
**What it does:** Drafts the actual reply to the customer, incorporating the classification and info-check results.
**Uses prior output:** Classification (Step 1) + missing info list (Step 2).

In [ ]:
# ---- STEP 3 PROMPT ----
STEP3_SYSTEM = """
You are a friendly, professional customer support agent for an e-commerce company.
Write a reply email to the customer addressing their issue.

Tone requirements:
- Empathetic and professional — acknowledge their frustration if sentiment is frustrated or angry.
- Solution-focused: always offer a concrete next step.
- Concise: max 150 words.
- Do NOT make promises you cannot keep (e.g., don't guarantee a specific delivery date).
- If information is missing, politely ask ONLY for what's listed in missing_info.

Format:
Subject: [subject line]

[email body]
"""

step3_input = f"""
Customer message: {customer_message}

Issue details:
- Category: {classification['category']}
- Urgency: {classification['urgency']}
- Sentiment: {classification['sentiment']}
- Summary: {classification['summary']}

Missing information to gather: {info_check['missing_info']}
Can we proceed without asking? {info_check['can_proceed']}
"""

step3_output = call_claude(STEP3_SYSTEM, step3_input, max_tokens=300)

print("=== STEP 3 OUTPUT (Customer Reply Draft) ===")
print(step3_output)

=== STEP 3 OUTPUT (Customer Reply Draft) ===
Subject: Re: Your Order #45821 – We're On It

Hi,

Thank you for reaching out, and I completely understand your frustration — waiting this long for something you need urgently for work is stressful, and I'm sorry for the inconvenience.

I've flagged Order #45821 as high priority and am escalating this directly with our shipping team to investigate why tracking hasn't updated in 5 days. We'll contact the carrier immediately to get a status update.

Here's what happens next:
- **Within 24 hours**, we'll provide you with a concrete update on your shipment's status.
- If the package is confirmed lost or significantly delayed, we'll arrange an expedited replacement or a full refund — whichever you prefer.

We know Monday is your deadline and we're doing everything we can to help.

We'll be in touch shortly. Thank you for your patience.

Warm regards,
Customer Support Team


---
## Step 4 — Escalation Decision
**What it does:** Reviews all prior context and decides whether this ticket should be escalated to a human specialist or closed.
**Uses prior output:** Classification (Step 1) + info check (Step 2) + draft reply (Step 3).

In [ ]:
# ---- STEP 4 PROMPT ----
STEP4_SYSTEM = """
You are a customer support supervisor AI. Review the full ticket context and decide on the correct action.

Escalation rules:
- ESCALATE if: urgency is 'high' AND sentiment is 'angry' or 'frustrated'
- ESCALATE if: order value appears to be large or business-critical
- ESCALATE if: issue cannot be resolved without internal system access
- CLOSE if: the drafted reply fully addresses the issue and no human follow-up is needed
- MONITOR if: reply was sent but awaiting customer response

Return JSON:
{
  "action": one of ["ESCALATE", "CLOSE", "MONITOR"],
  "reason": one sentence explaining the decision,
  "escalate_to": one of ["shipping_team", "billing_team", "senior_support", "none"]
}

Output ONLY valid JSON.
"""

step4_input = f"""
Original customer message: {customer_message}

Classification:
- Category: {classification['category']}
- Urgency: {classification['urgency']}
- Sentiment: {classification['sentiment']}

Missing info identified: {info_check['missing_info']}

Drafted reply:
{step3_output}
"""

step4_output = call_claude(STEP4_SYSTEM, step4_input)

print("=== STEP 4 OUTPUT (Escalation Decision) ===")
print(step4_output)

escalation = json.loads(step4_output)
print("\nFinal action:", escalation['action'])
print("Reason:", escalation['reason'])
print("Escalate to:", escalation['escalate_to'])

=== STEP 4 OUTPUT (Escalation Decision) ===
{
  "action": "ESCALATE",
  "reason": "The ticket meets escalation criteria with high urgency and frustrated sentiment, and resolving the stalled shipment requires internal system access to contact the carrier and investigate Order #45821.",
  "escalate_to": "shipping_team"
}

Final action: ESCALATE
Reason: The ticket meets escalation criteria with high urgency and frustrated sentiment, and resolving the stalled shipment requires internal system access to contact the carrier and investigate Order #45821.
Escalate to: shipping_team


---
## Full Chain Summary

In [ ]:
print("==========================================")
print("        PROMPT CHAIN FULL SUMMARY")
print("==========================================")
print(f"\n[STEP 1] Classification:")
print(f"  Category : {classification['category']}")
print(f"  Urgency  : {classification['urgency']}")
print(f"  Sentiment: {classification['sentiment']}")
print(f"  Summary  : {classification['summary']}")

print(f"\n[STEP 2] Missing Info:")
for item in info_check['missing_info']:
    print(f"  - {item}")
print(f"  Can proceed: {info_check['can_proceed']}")

print(f"\n[STEP 3] Customer Reply Draft:")
print(step3_output)

print(f"\n[STEP 4] Escalation Decision:")
print(f"  Action     : {escalation['action']}")
print(f"  Reason     : {escalation['reason']}")
print(f"  Escalate to: {escalation['escalate_to']}")
print("\n========================================== END")

        PROMPT CHAIN FULL SUMMARY

[STEP 1] Classification:
  Category : shipping
  Urgency  : high
  Sentiment: frustrated
  Summary  : Laptop order #45821 stuck in transit for 5 days and needed urgently by Monday.

[STEP 2] Missing Info:
  - Confirm shipping carrier handling the delivery
  - Confirm delivery address on file is correct
  Can proceed: True

[STEP 3] Customer Reply Draft:
Subject: Re: Your Order #45821 – We're On It

Hi,

Thank you for reaching out, and I completely understand your frustration — waiting this long for something you need urgently for work is stressful, and I'm sorry for the inconvenience.

I've flagged Order #45821 as high priority and am escalating this directly with our shipping team to investigate why tracking hasn't updated in 5 days. We'll contact the carrier immediately to get a status update.

Here's what happens next:
- **Within 24 hours**, we'll provide you with a concrete update on your shipment's status.
- If the package is confirmed lost or si

---
## Iteration — Before vs After (Prompt Improvement)

### Step 1 Prompt — v1 (original, had problems)

**Prompt used:**
```
System: "You are a customer support triage assistant.
         Read the customer message and classify it into a category."

User: "Customer message: {customer_message}"
```

**Problem — sample v1 output (prose, not parseable):**
```
This appears to be a shipping issue. The customer seems frustrated
because their order hasn't arrived. Urgency seems high since they
need it for work. I would classify this as a delivery problem.
```
**Issue:** Returned free-form prose — `json.loads()` threw a `JSONDecodeError`. No structured fields.

---

### Step 1 Prompt — v2 (fixed)

**Changes made:** Added explicit JSON schema, fixed field names, added hard output constraint.

```
System: "You are a customer support triage assistant. Your ONLY job is to classify
         incoming support messages.

         Return a JSON object with EXACTLY these fields:
         {
           'category': one of ['shipping','billing','technical','returns','account','other'],
           'urgency':  one of ['low','medium','high'],
           'sentiment': one of ['positive','neutral','frustrated','angry'],
           'summary': a single sentence (max 20 words) summarising the core issue
         }

         Rules:
         - Output ONLY valid JSON. No markdown fences, no explanations.
         - Do not invent information not present in the message."

User: "Customer message: {customer_message}"
```

**v2 output (valid JSON, parseable):**
```json
{
  "category": "shipping",
  "urgency": "high",
  "sentiment": "frustrated",
  "summary": "Customer's laptop order delayed 10 days with stale tracking info."
}
```
**Fix worked:** `json.loads()` succeeds; structured data flows cleanly into Step 2.

---

### Step 3 Prompt — v1 vs v2

| | v1 (original) | v2 (fixed) |
|---|---|---|
| Word limit | None specified | `max 150 words` added |
| v1 problem | Reply was 240+ words, too long for a support ticket | |
| Fix | Added explicit word constraint | |

### Step 4 Prompt — v1 vs v2

| | v1 (original) | v2 (fixed) |
|---|---|---|
| Escalation rules | Vague: "escalate if needed" | Explicit bullet conditions added |
| v1 problem | Returned ESCALATE for low-urgency tickets | |
| Fix | Added `urgency is high AND sentiment is frustrated/angry` rule | |